In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# train, test 분리
from sklearn.model_selection import train_test_split

#RandomForest
from sklearn.ensemble import RandomForestRegressor

# GridSearchCV : 하이퍼파라미터 설정
from sklearn.model_selection import GridSearchCV

In [ ]:
# continuous_factory_process 파일 불러오기
raw_df = pd.read_csv('./data/continuous_factory_process.csv', index_col="time_stamp")

# Setpoint 제거
del_columns = [col for col in raw_df.columns if 'Setpoint' in col]
raw_df.drop(del_columns, axis=1, inplace=True)

In [ ]:
# continuous_factory_process 파일 불러오기
sub_df = pd.read_csv('./data/submission_data.csv', index_col="time_stamp")

# Setpoint 제거
del_columns = [col for col in sub_df.columns if 'Setpoint' in col]
sub_df.drop(del_columns, axis=1, inplace=True)

In [ ]:
model1_X = raw_df.loc[:, :'FirstStage.CombinerOperation.Temperature3.C.Actual']
model1_y = raw_df.loc[:,'Stage1.Output.Measurement0.U.Actual':'Stage1.Output.Measurement14.U.Actual']
model2_X = raw_df.loc[:,'Machine4.Temperature1.C.Actual':'Machine5.ExitTemperature.U.Actual']
model2_y = raw_df.loc[:,'Stage2.Output.Measurement0.U.Actual':'Stage2.Output.Measurement14.U.Actual']

subdata_X1 = sub_df.loc[:, :'FirstStage.CombinerOperation.Temperature3.C.Actual']
subdata_X2 = sub_df.loc[:,'Machine4.Temperature1.C.Actual':'Machine5.ExitTemperature.U.Actual']

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(model1_X,model1_y,test_size=0.3, random_state=10)

In [ ]:
# GridSearchCV 하이퍼 파라미터 튜닝

# 1. 튜닝할 하이퍼파라미터 설정
param = {'n_estimators' : [50, 150, 250, 350, 450, 1000], #나무 개수
         'max_depth' : [5, 7, 9, 11, 15, 20], #나무 최대 깊이
         'max_leaf_nodes' : [3, 5, 7], #최대 리프 노드 수
         'min_samples_split' : [20,40,60]} #노드 분할을 위한 최소 샘플 수

# 2. 랜덤 포레스트 모델 생성
forest_model1 = RandomForestRegressor(random_state=10, n_jobs=-1)

# 3. GridSearchCV 모델 생성
grid = GridSearchCV(forest_model1, param, cv=5, scoring = 'accuracy', n_jobs=-1)


# 4. grid 모델 학습
grid.fit(X1_train, y1_train)

In [ ]:
# 모델 생성 및 하이퍼 파라미터 튜닝
forest_model1 = RandomForestRegressor(n_estimators=100, random_state=10)
forest_model2 = RandomForestRegressor(n_estimators=100, random_state=10)

In [ ]:
# 모델1 학습
forest_model1.fit(model1_X, model1_y)

In [ ]:
# 모델2 학습
forest_model2.fit(model2_X, model2_y)

In [ ]:
X1_pred = forest_model1.predict(subdata_X1)
X2_pred = forest_model2.predict(subdata_X2)

In [ ]:
# X1_pred를 'submission1.npy' 파일로 저장
np.save('submission1.npy', X1_pred)
np.save('submission2.npy', X2_pred)

### XGBoost 모델

In [1]:
# continuous_factory_process 파일 불러오기
raw_df = pd.read_csv('./data/continuous_factory_process.csv', index_col="time_stamp")

# Setpoint 제거
del_columns = [col for col in raw_df.columns if 'Setpoint' in col]
raw_df.drop(del_columns, axis=1, inplace=True)

# continuous_factory_process 파일 불러오기
sub_df = pd.read_csv('./data/submission_data.csv', index_col="time_stamp")

# Setpoint 제거
del_columns = [col for col in sub_df.columns if 'Setpoint' in col]
sub_df.drop(del_columns, axis=1, inplace=True)

model1_X = raw_df.loc[:, :'FirstStage.CombinerOperation.Temperature3.C.Actual']
model1_y = raw_df.loc[:,'Stage1.Output.Measurement0.U.Actual':'Stage1.Output.Measurement14.U.Actual']
model2_X = raw_df.loc[:,'Machine4.Temperature1.C.Actual':'Machine5.ExitTemperature.U.Actual']
model2_y = raw_df.loc[:,'Stage2.Output.Measurement0.U.Actual':'Stage2.Output.Measurement14.U.Actual']

subdata_X1 = sub_df.loc[:, :'FirstStage.CombinerOperation.Temperature3.C.Actual']
subdata_X2 = sub_df.loc[:,'Machine4.Temperature1.C.Actual':'Machine5.ExitTemperature.U.Actual']

from xgboost import XGBClassifier

# 모델생성 및 하이퍼파라미터 설정
xgb_model1 = XGBClassifier(random_state=10)

import xgboost as xgb

## 모델1
from xgboost import XGBRegressor

# 15개의 XGBoost 회귀 모델을 저장할 리스트 생성
xgb_models = []

# 각 출력 특성을 개별적으로 학습
for i in range(15):
    xgb_model = XGBRegressor(n_estimators=100, random_state=10)
    xgb_model.fit(model1_X, model1_y[f'Stage1.Output.Measurement{i}.U.Actual'])
    xgb_models.append(xgb_model)

# subdata_X1에 대한 예측을 저장할 리스트 생성
X1_pred = []

# 각 XGBoost 모델에 대해 예측 수행
for xgb_model in xgb_models:
    pred = xgb_model.predict(subdata_X1)
    X1_pred.append(pred)

# X1_pred 리스트를 NumPy 배열로 변환
X1_pred = np.array(X1_pred).T  # Transpose for shape (n_samples, 15)

## 모델2

# 15개의 XGBoost 회귀 모델을 저장할 리스트 생성
xgb_models2 = []

# 각 출력 특성을 개별적으로 학습
for i in range(15):
    xgb_model2 = XGBRegressor(n_estimators=100, random_state=10)
    xgb_model2.fit(model2_X, model2_y[f'Stage2.Output.Measurement{i}.U.Actual'])
    xgb_models2.append(xgb_model2)

# subdata_X1에 대한 예측을 저장할 리스트 생성
X2_pred = []

# 각 XGBoost 모델에 대해 예측 수행
for xgb_model2 in xgb_models2:
    pred2 = xgb_model2.predict(subdata_X2)
    X2_pred.append(pred2)

# X1_pred 리스트를 NumPy 배열로 변환
X2_pred = np.array(X2_pred).T  # Transpose for shape (n_samples, 15)

NameError: name 'pd' is not defined